<a href="https://colab.research.google.com/github/AndreDG88/anotacoes_python_curso_ebac/blob/main/python_avancado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🐍 Python Avançado**


## **⚙️ Iteradores, yield, Geradores, Corrotinas, Assíncrono e Multiprocessamento**

### **1. O que é um Iterador?**

**Definição Simples:**

Um **iterador** é um objeto que você pode percorrer **item por item**, como numa lista, usando um *for*.

In [ ]:
# Exemplo:

nomes = ["Ana", "Beto", "Clara"]

for nome in nomes:
    print(nome)

Ana
Beto
Clara


Aqui, nomes é **iterável** (porque dá pra percorrer com for), e o que o *for* faz internamente é usar um **iterador** para pegar um nome de cada vez.

In [ ]:
# De forma explícita:

nomes = ["Ana", "Beto", "Clara"]
it = iter(nomes)     # Cria o iterador

print(next(it))  # Ana
print(next(it))  # Beto
print(next(it))  # Clara

Ana
Beto
Clara


Se tentar chamar **next(it)** de novo, vai dar erro (StopIteration) porque não tem mais nada.

### **2. yield e Geradores (Generators)**

**Como funciona:**

Quando usamos **yield**, estamos criando uma **função geradora**, que se comporta como um iterador, mas gera os valores um por um.

In [ ]:
# Exemplo:

def contador():
    yield 1
    yield 2
    yield 3

for numero in contador():
    print(numero)

1
2
3


Essa função **"lembra onde parou"** entre cada chamada. É isso que faz ela economizar memória.

### **3. Corrotinas com yield e .send()**

**Ideia principal:**
Uma **corrotina** com *yield* pode **receber valores** durante a execução.

In [ ]:
# Exemplo:

def acumulador():
    total = 0
    while True:
        valor = yield total
        if valor is not None:
            total += valor

coro = acumulador()
print(next(coro))      # Começa a execução, imprime 0
print(coro.send(5))    # Envia 5, imprime 5
print(coro.send(10))   # Envia 10, imprime 15

0
5
15


### **4. Programação Assíncrona (async e await)**

**Quando usar:**
- Quando você quer esperar algo (como ler arquivos ou acessar a internet), mas sem travar o resto do programa.

**Termos:**
- async def define uma função assíncrona (corrotina moderna)

- await espera uma tarefa terminar antes de continuar

In [ ]:
# Exemplo:

import asyncio

async def esperar(msg):
    print(f"Iniciando {msg}")
    await asyncio.sleep(2)
    print(f"Finalizando {msg}")

async def main():
    await asyncio.gather(
        esperar("um café ☕"),
        esperar("uma música 🎵")
    )

asyncio.run(main())

Executa as duas ao mesmo tempo (concorrente) e finaliza em 2 segundos.

### **5. Multiprocessamento (multiprocessing)**

**Quando usar:**

- Para tarefas **pesadas de CPU**, onde rodar várias ao mesmo tempo pode acelerar o processo.

- Usa **vários processos** (não threads), então ignora o GIL (trava do Python).

In [ ]:
# Exemplo:

from multiprocessing import Process

def tarefa(nome):
    print(f"{nome} começou.")

p1 = Process(target=tarefa, args=("Processo 1",))
p2 = Process(target=tarefa, args=("Processo 2",))

p1.start()
p2.start()

p1.join()
p2.join()

Cada processo é independente e pode rodar em paralelo em diferentes núcleos.

### **✅ Resumo Visual**

In [ ]:
#Conceito          | Para que serve?                     | Palavra-chave  | Exemplo típico
Iterador           | Percorrer elementos                 | iter(), next() | for item in lista:
yield              | Pausar e retomar funções            | yield          | for x in gerador():
Gerador            | Economizar memória em laços         | yield          | def gen(): yield ...
Corrotina (yield)  | Receber dados durante execução      | .send()        | valor = yield total
Async/Await        | Esperar tarefas sem travar o código | async, await   | await asyncio.sleep(1)
Multiprocessos     | Rodar tarefas pesadas em paralelo   | Process        | from multiprocessing import Process

## **⚙️ CPU-Bound, I/O-Bound e GIL**

### **1. O que é CPU-bound?**

**Explicação:**

"CPU-bound" é um termo usado para descrever programas ou partes do código que **dependem fortemente do poder de processamento da CPU** para funcionar.

Ou seja: o computador **está ocupado fazendo cálculos pesados**, e isso consome tempo do processador.

**Analogia:**

Imagine um chef cortando legumes: ele não pode parar nem dividir isso com alguém, porque só ele tem a faca. Enquanto ele corta, ninguém mais consegue usar a cozinha.

🧠 **Exemplos práticos:**
- Cálculos matemáticos pesados

- Processamento de imagens ou vídeos

- Compressão ou criptografia de dados

🧪 **Exemplo em código:**

In [ ]:
# Faz uma conta bem pesada
def tarefa_pesada():
    total = 0
    for i in range(10**8):
        total += i
    return total

print(tarefa_pesada())

4999999950000000


### **2. O que é I/O-bound?**

**Explicação:**

"I/O-bound" se refere a situações em que seu código **fica esperando alguma entrada ou saída de dados (Input/Output)**. Nesses casos, a CPU fica ociosa, parada, enquanto o programa aguarda algo.

**Analogia:**

Imagine que o chef fez uma massa e colocou pra assar. Enquanto o forno trabalha, ele está ali esperando. Ele **não está fazendo nada**, só aguardando.

🧠 **Exemplos práticos:**
- Aguardar leitura ou gravação de arquivos

- Baixar dados da internet

- Esperar o usuário digitar algo

🧪 **Exemplo em código:**

In [ ]:
import time

def ler_site():
    print("Acessando site...")
    time.sleep(3)  # Simula o tempo de espera pela internet
    print("Dados recebidos.")

print(ler_site())

Acessando site...
Dados recebidos.
None


### **3. O que é GIL (Global Interpreter Lock)?**

**Explicação:**

O **GIL** é uma trava interna do Python (especificamente da versão mais comum, chamada **CPython**). Ele garante que **apenas uma thread execute código Python por vez**, mesmo em computadores com múltiplos núcleos.

Isso **evita problemas de concorrência** (dois pedaços de código tentando acessar os mesmos dados ao mesmo tempo), mas também **limita o desempenho em tarefas CPU-bound**.

**Analogia:**

Pense em um restaurante com vários cozinheiros, mas **só um pode usar a frigideira de cada vez**. Mesmo tendo vários funcionários, você perde desempenho se todos precisarem usar a mesma coisa ao mesmo tempo.

🧠 **Por que isso importa?**
- Em tarefas leves ou que envolvem espera (I/O-bound), o GIL não atrapalha muito.

- Em tarefas de cálculo pesado, o GIL pode se tornar um problema de desempenho se você estiver usando threads.

## **⚙️ Threads e Multiprocessing**

### **4. O que são Threads? (Multithreading)**

**Explicação:**

Threads **são pequenas unidades de execução** que rodam "ao mesmo tempo" dentro do mesmo programa. A ideia é fazer várias tarefas em paralelo, **compartilhando a memória do processo principal**.

**Analogia:**

Como várias pessoas trabalhando ao mesmo tempo na mesma cozinha, mas dividindo os mesmos ingredientes e utensílios.

🧠 **Ideal para:**

Programas que esperam coisas externas (I/O-bound), como:

- Aguardar resposta da internet

- Esperar leitura de arquivos

⚠️ **Cuidado:**

- Não serve bem para tarefas CPU-bound por causa do GIL.

- Pode gerar erros difíceis de encontrar se várias threads mexerem nos mesmos dados ao mesmo tempo.

🧪 **Exemplo em código:**

In [ ]:
import threading
import time

def tarefa():
    print("Tarefa começou")
    time.sleep(2)
    print("Tarefa terminou")

t1 = threading.Thread(target=tarefa)
t2 = threading.Thread(target=tarefa)

t1.start()
t2.start()
t1.join()
t2.join()


Tarefa começou
Tarefa começou
Tarefa terminou
Tarefa terminou


### **5. O que é Multiprocessing?**

**Explicação:**

multiprocessing é uma forma de rodar **vários processos separados**, cada um com sua própria memória e **sem depender do GIL**. É como abrir vários programas Python ao mesmo tempo, cada um cuidando de uma parte do trabalho.

**Analogia:**

Agora cada cozinheiro tem **sua própria cozinha**: ninguém atrapalha ninguém. Ideal para tarefas pesadas!

🧠 **Ideal para:**

- Programas que fazem cálculos pesados (CPU-bound)

- Dividir trabalho entre núcleos da CPU

🧪 **Exemplo:**

In [ ]:
from multiprocessing import Process

def tarefa_pesada():
    total = sum(range(10**7))
    print(f"Resultado: {total}")

p1 = Process(target=tarefa_pesada)
p2 = Process(target=tarefa_pesada)

p1.start()
p2.start()
p1.join()
p2.join()

Resultado: 49999995000000Resultado: 49999995000000



## **⚙️ O que é TDD (Test Driven Development)?**

**Explicação:**

**TDD** significa "Desenvolvimento Guiado por Testes". É uma técnica onde você **escreve o teste primeiro**, antes de escrever o código da função.

O objetivo é garantir que **o que você está escrevendo faz exatamente o que deveria fazer**.

**Analogia:**

É como escrever o gabarito antes de fazer a prova: você sabe exatamente o que é esperado antes de começar.

🧠 **Por que usar TDD?**

- Ajuda a organizar o código

- Garante que não quebre nada sem querer

- Dá segurança para mudar e melhorar o código depois

✅ **Um modelo é TDD se seguir essa sequência:**

- 1- Você escreve os testes primeiro, sem ter ainda implementado as funções.

- 2- Ao rodar os testes, eles falham, porque as funções ainda nem existem (isso é esperado!).

- 3- Aí você vai implementando o código aos poucos, até os testes passarem.

- 4- Por fim, você refatora (melhora) o código, mantendo os testes passando.



🧪 **Exemplo simples com pytest:** É mais moderno e simples de usar. Você escreve testes como funções comuns.

In [ ]:
# Primeiro escrevemos o teste:
def test_somar():
    assert somar(2, 3) == 5

# Depois implementamos a função:
def somar(a, b):
    return a + b

Para rodar o teste, basta usar o pytest:

In [ ]:
$ pytest nome_do_arquivo.py

🧪 Exemplo simples com unittest: Vem com o Python, mais "formal", baseado em classes.

In [ ]:
# escrevendo os testes primeiro:
import unittest
from operacoes import eh_par  # a função ainda não existe!

class TestOperacoes(unittest.TestCase):
    def test_numero_par(self):
        self.assertTrue(eh_par(4))  # 4 é par

    def test_numero_impar(self):
        self.assertFalse(eh_par(5))  # 5 é ímpar

if __name__ == '__main__':
    unittest.main()

Tenta rodar esse teste

In [ ]:
python test_operacoes.py

🔴 Vai dar erro! A função eh_par ainda não existe. Isso é esperado no TDD.

**Cria o código mínimo para passar no teste**

Crie um arquivo operacoes.py com a função que falta:

In [ ]:
def eh_par(numero):
    return numero % 2 == 0

Roda os testes novamente

In [ ]:
python test_operacoes.py

✅ Agora os testes passam!

**(Opcional) Refatoração**

Nesse caso, o código já está limpo, mas se estivesse complicado, a ideia agora seria melhorá-lo **sem quebrar os testes**.

🧪 **Comparação: unittest vs pytest**

In [ ]:
	             unittest	                 pytest
Estilo	     | Classes com métodos	   | Funções simples
Verbosidade	 | Mais verboso	           | Mais direto
Popularidade | Muito usado em empresas | Muito usado em projetos novos
TDD?	       | ✅ Suporta TDD          | ✅ Também

## **⚙️ Binary Search (Busca Binária)**

A **Busca Binária** é um algoritmo super rápido para procurar um valor dentro de uma **lista ordenada**.

**Ela funciona assim:**

- 1- Pega o elemento do meio da lista.

- 2- Se for o que você quer, acabou!

- 3- Se o número for maior, ignora a metade de baixo.

- 4- Se for menor, ignora a metade de cima.

- 5- Repete até encontrar ou a lista acabar.

🔎 **Analogia:**

Pense em um dicionário. Se você quer encontrar a palavra "morango", você **não vai folhear página por página**. Você vai no meio, vê se está perto, e vai afinando até achar.

🧠 **Exemplo prático:**

Vamos procurar o número 42 nesta lista ordenada:

In [ ]:
numeros = [10, 18, 22, 33, 42, 57, 63, 71, 89]

Código com resolução comentada:

In [ ]:
def busca_binaria(lista, alvo):
    inicio = 0
    fim = len(lista) - 1

    while inicio <= fim:
        meio = (inicio + fim) // 2 # calcula o índice do meio
        chute = lista[meio] # valor do meio da lista

        print(f"Verificando índice {meio}: {chute}")

        if chute == alvo: # achou!
            return meio
        elif chute < alvo: # ignora a metade de baixo
            inicio = meio + 1
        else: # ignora a metade de cima
            fim = meio - 1

    return -1  # se não encontrar

# código para testar a função
if __name__ == '__main__':
    numeros = [10, 18, 22, 33, 42, 57, 63, 71, 89]
    procurado = 42
    resultado = busca_binaria(numeros, procurado)

    if resultado != -1:
        print(f"Número {procurado} encontrado no índice {resultado}")
    else:
        print("Número não encontrado.")

Verificando índice 4: 42
Número 42 encontrado no índice 4


▶️ **Como executar:**

Abra o terminal, vá até a pasta onde está o arquivo, e rode:

In [ ]:
python3 busca_binaria.py

**Explicando:**
- lista → é a lista onde vamos procurar.

- alvo → é o número que queremos encontrar.

- inicio, fim → são os limites da parte da lista que estamos olhando.

- meio → calcula a posição central da lista atual.

- while → repete até encontrar ou passar do limite.

- return → devolve a posição onde encontrou (ou -1 se não encontrou).

⚠️ **Lembrete importante:**

A **lista precisa estar ordenada** pra usar Binary Search. Se não estiver, o resultado vai dar errado.

## **⚙️ Bubble Sort (Ordenação por Bolha)**

É um algoritmo de ordenação simples que **compara os elementos aos pares** e os troca de lugar **se estiverem fora de ordem**.

Esse processo **se repete várias vezes**, até que a lista esteja ordenada.

**Analogia:**

Pensa em bolhas subindo na água: os valores grandes vão "subindo" pro fim da lista como bolhas.

🧠 **Exemplo:**

Vamos ordenar esta lista:

In [ ]:
numeros = [5, 1, 4, 2, 8]

Código com resolução comentada:

In [ ]:
def bubble_sort(lista):
    n = len(lista)
    for i in range(n):
        for j in range(0, n-i-1): # percorre até o final da parte ainda não ordenada
            if lista[j] > lista[j+1]:
                # Troca se o da esquerda for maior que o da direita
                lista[j], lista[j+1] = lista[j+1], lista[j]
                print(f"Troca: {lista}")
    return lista

# código para testar a função
if __name__ == '__main__':
    numeros = [5, 1, 4, 2, 8]
    print("Lista original:", numeros)
    ordenada = bubble_sort(numeros)
    print("Lista ordenada:", ordenada)

Lista original: [5, 1, 4, 2, 8]
Troca: [1, 5, 4, 2, 8]
Troca: [1, 4, 5, 2, 8]
Troca: [1, 4, 2, 5, 8]
Troca: [1, 2, 4, 5, 8]
Lista ordenada: [1, 2, 4, 5, 8]


▶️ **Como executar:**

No terminal, rode:

In [ ]:
python3 bubble_sort.py

🧠 **Explicando:**

- for i in range(n) → controla quantas passagens serão feitas.

- for j in range(0, n - i - 1) → compara os pares de elementos.

- if lista[j] > lista[j+1] → se o da esquerda for maior, troca com o da direita.

- lista[j], lista[j+1] = lista[j+1], lista[j] → essa é a "troca".

📚 **Dica Final: Como testar com outros valores**

Você pode mudar a lista de números ou pedir para o usuário digitar, por exemplo:

In [ ]:
numeros = list(map(int, input("Digite números separados por espaço: ").split()))